In [2]:
# imports packages
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import scipy.integrate as integrate
from numpy.polynomial import Polynomial as P
from cgmbrush import *

## Testing


RS_array = np.zeros(9)
RS_array[0] = CDtoRS(L,1)
for i in range(1,len(RS_array)):
    RS_array[i]= z_eff(CDtoRS(float(L*i),2),CDtoRS(float(L*(i+1)),2),L)
    
    

z_eff(CDtoRS(L*3,1),CDtoRS(L*4,1),L)

len(RS_array_gen(1,L))

elecD(0)*L*10**6

L

Lbox = int_zmin^zmax  c dz/H(z)
Lbox~ cdz/H(z)


Dconf(1)-Dconf(0.865)

CDtoRS(250*15,1)

CDtoRS(357,1)

CDtoRS(L,1)

CD